In [1]:
from glob import glob
from tools.procOps import *
from tools.fileOps import *

In [2]:
import os
os.chdir('/public/groups/cgl/cat/primates_evan/')

In [3]:
flnc = glob('isoseq/*/*fasta')

In [5]:
cfg = [x.split() for x in open('assemblies/cactus.config')][2:]
d = {x[0].lower().replace('*', ''): x[1].split('evan/')[1] for x in cfg}

In [35]:
for f in flnc:
    g = f.split('/')[1].split('_')[0]
    a = d[g]
    out = f.replace('.fasta', '.sorted.bam')
    cmd = [['minimap2', '-t', '12', '-ax', 'splice:hq', '-uf', a, f],
            ['samtools', 'view', '-bS', '-'],
            ['sambamba', 'sort', '/dev/stdin', '-o', out]]
    run_proc(cmd)

In [6]:
# repeat for old assemblies
old_fastas = glob('/public/groups/cgl/cat/original_primates/genome_files/*fa')
old_d = {}
for fasta in old_fastas:
    genome = os.path.basename(fasta).split('.')[0].lower()
    if genome in d and genome != 'human':
        old_d[genome] = fasta

In [7]:
out_dir = '/public/groups/cgl/cat/primates_evan/isoseq_original_primates'
for f in flnc:
    g = f.split('/')[1].split('_')[0]
    if g not in old_d:
        continue
    a = old_d[g]
    out = f.replace('.fasta', '.sorted.bam')
    out = out.replace('isoseq', out_dir, 1)
    !mkdir -p {os.path.dirname(out)}
    cmd = [['minimap2', '-t', '12', '-ax', 'splice:hq', '-uf', a, f],
            ['samtools', 'view', '-bS', '-'],
            ['sambamba', 'sort', '/dev/stdin', '-o', out]]
    run_proc(cmd)